# Doawnloads and Stores IPO S-1 Filings as .txt Files

* Uses EDGAR search
 * https://www.sec.gov/edgar/searchedgar/companysearch.html
* Uses NASDAQ IPO lists
 * https://www.nasdaq.com/markets/ipos/
* Skips ambiguous company names (do manually)
* Takes latest file if multiple files are available (S-1, S-1/A etc.)

In [1]:
import edgar
import nasdaq

from pathlib import Path

### Scrape NASDAQ IPO Lists

In [2]:
df_symbols = nasdaq.get_ipo_list('2000-01-01')

date range: odict_keys(['2000-01', '2000-02', '2000-03', '2000-04', '2000-05', '2000-06', '2000-07', '2000-08', '2000-09', '2000-10', '2000-11', '2000-12', '2001-01', '2001-02', '2001-03', '2001-04', '2001-05', '2001-06', '2001-07', '2001-08', '2001-09', '2001-10', '2001-11', '2001-12', '2002-01', '2002-02', '2002-03', '2002-04', '2002-05', '2002-06', '2002-07', '2002-08', '2002-09', '2002-10', '2002-11', '2002-12', '2003-01', '2003-02', '2003-03', '2003-04', '2003-05', '2003-06', '2003-07', '2003-08', '2003-09', '2003-10', '2003-11', '2003-12', '2004-01', '2004-02', '2004-03', '2004-04', '2004-05', '2004-06', '2004-07', '2004-08', '2004-09', '2004-10', '2004-11', '2004-12', '2005-01', '2005-02', '2005-03', '2005-04', '2005-05', '2005-06', '2005-07', '2005-08', '2005-09', '2005-10', '2005-11', '2005-12', '2006-01', '2006-02', '2006-03', '2006-04', '2006-05', '2006-06', '2006-07', '2006-08', '2006-09', '2006-10', '2006-11', '2006-12', '2007-01', '2007-02', '2007-03', '2007-04', '2007-05

C:\Users\Void\Anaconda3\lib\site-packages\pandas\core\indexes\api.py:87: RuntimeWarning: '<' not supported between instances of 'str' and 'int', sort order is undefined for incomparable objects
  result = result.union(other)
C:\Users\Void\Anaconda3\lib\site-packages\pandas\core\indexes\api.py:87: RuntimeWarning: '<' not supported between instances of 'int' and 'str', sort order is undefined for incomparable objects
  result = result.union(other)


In [3]:
df_symbols.head(3)

,Company Name,Date Priced,Market,Offer Amount,Price,Shares,Symbol,0,1,2
Symbol,,,,,,,,,,
APL,TARGA PIPELINE PARTNERS LP,2000-01-28,American Stock Exchange,19500000.0,13.0,1.5e+06,APL,NaN,NaN,NaN
ASPM,ASPECT MEDICAL SYSTEMS INC,2000-01-28,NASDAQ,52500000.0,15.0,3.5e+06,ASPM,NaN,NaN,NaN
CAMZ,CAMINUS CORP,2000-01-28,NASDAQ,69952000.0,16.0,4.372e+06,CAMZ,NaN,NaN,NaN


### Batch Download

In [4]:
#batch download
counter = 0

for x in df_symbols['Symbol']:
    try:
        counter += 1
        print('\n( ' + str(counter) + ' / ' + str(df_symbols.shape[0]) + ' ) ' + str(x))

        #check if exists
        if Path("../Data/" + x + ".htm").is_file():
            print(x + ' data already exists, skipping...')
            continue

        #create company
        tmpEdgar = edgar.Edgar()
        possible_companies = tmpEdgar.findCompanyName(df_symbols.loc[x]['Company Name'])
        print('possible_companies:', possible_companies)

        #validate
        if len(possible_companies) == 0:
            print('no possible companies:', x)
            continue

        name = possible_companies[0]
        cik = tmpEdgar.getCikByCompanyName(possible_companies[0])
        company = edgar.Company(name, cik)

        #look for S-1
        tree = company.getAllFilings(filingType = "S-1")
        docs = edgar.getDocuments(tree, noOfDocuments=5)

        #if multiple amendments exist
        #filter ones with length lower than 500K (statistically the mean is higher)
        docs = [d for d in docs if len(d) > 500000]

        #write file
        if len(docs) > 0:
            with open("../Data/" + x + ".htm", "w", encoding="utf-8") as f:
                f.write(docs[0].decode("utf-8"))
                print('Got data for ' + x + ' chose ' + str(len(docs[0])))
    except Exception as e:
        print(x, e)


( 1 / 3665 ) APL
possible_companies: ['TARGA PIPELINE PARTNERS LP']
doc 348472 https://www.sec.gov/Archives/edgar/data/1092914/000119312509203562/ds1a.htm
doc 339088 https://www.sec.gov/Archives/edgar/data/1092914/000119312509187003/ds1.htm
doc 335798 https://www.sec.gov/Archives/edgar/data/1092914/000119312509132490/ds1.htm
doc 11722 https://www.sec.gov/Archives/edgar/data/1092914/000095011600000079/
doc 10662 https://www.sec.gov/Archives/edgar/data/1092914/000095011600000010/

( 2 / 3665 ) ASPM
possible_companies: ['ASPECT MEDICAL SYSTEMS INC']
doc 10302 https://www.sec.gov/Archives/edgar/data/886235/000095013500000274/
doc 11183 https://www.sec.gov/Archives/edgar/data/886235/000095013500000272/
doc 10477 https://www.sec.gov/Archives/edgar/data/886235/000095013500000258/
doc 12064 https://www.sec.gov/Archives/edgar/data/886235/000095013500000197/
doc 10830 https://www.sec.gov/Archives/edgar/data/886235/000095013500000061/

( 3 / 3665 ) CAMZ
possible_companies: ['CAMINUS CORP']

( 4 

doc 10484 https://www.sec.gov/Archives/edgar/data/1091953/000095013000000725/
doc 11014 https://www.sec.gov/Archives/edgar/data/1091953/000095013000000499/
doc 10836 https://www.sec.gov/Archives/edgar/data/1091953/000095013000000243/
doc 11546 https://www.sec.gov/Archives/edgar/data/1091953/000095013000000006/
doc 14378 https://www.sec.gov/Archives/edgar/data/1091953/000095013099006664/

( 32 / 3665 ) APRS
possible_companies: ['APROPOS TECHNOLOGY INC']
doc 10309 https://www.sec.gov/Archives/edgar/data/1098803/000095013100001295/
doc 10309 https://www.sec.gov/Archives/edgar/data/1098803/000095013100001262/
doc 11371 https://www.sec.gov/Archives/edgar/data/1098803/000095013100000994/
doc 10662 https://www.sec.gov/Archives/edgar/data/1098803/000095013100000480/
doc 11900 https://www.sec.gov/Archives/edgar/data/1098803/000095013100000357/

( 33 / 3665 ) JCWW
possible_companies: ['LIFEQUEST WORLD CORP']
doc 10308 https://www.sec.gov/Archives/edgar/data/1060888/000089710199000601/
doc 33958 

possible_companies: ['FN ESTATE INC']
doc 10659 https://www.sec.gov/Archives/edgar/data/1092536/000091205700004301/
doc 15262 https://www.sec.gov/Archives/edgar/data/1092536/000091205700003587/
doc 10661 https://www.sec.gov/Archives/edgar/data/1092536/000091205700001455/
doc 10484 https://www.sec.gov/Archives/edgar/data/1092536/000091205799011119/
doc 10661 https://www.sec.gov/Archives/edgar/data/1092536/000091205799009036/

( 60 / 3665 ) BUYX
possible_companies: ['BIDORBUY.COM INC', 'BUY COM INC', 'BUY IT CHEAP COM INC /DE', 'BUYERSONLINECOM INC', 'BUYERZONE COM INC', 'BUYIT COM INC', 'EATON VANCE TAX-MANAGED BUY-WRITE INCOME FUND', 'EATON VANCE TAX-MANAGED BUYWRITE INCOME FUND', 'HOTBUYER COM INC', 'IBUYDIGITAL.COM, INC.', 'MEDIBUY COM INC', 'NUVEEN S&P 500 BUYWRITE INCOME FUND', 'SHOEBUY COM INC', 'TELCOBUY COM INC', 'UBUYJAWS COM INC', 'WILSON BUY-WRITE INCOME FUND, LP']

( 61 / 3665 ) AMWW
possible_companies: ['AIMS WORLDWIDE INC']

( 62 / 3665 ) AGEN
possible_companies: ['AGENUS 

doc 11367 https://www.sec.gov/Archives/edgar/data/1100983/000091205700002846/
doc 13839 https://www.sec.gov/Archives/edgar/data/1100983/000091205799009843/

( 84 / 3665 ) TLCT
possible_companies: ['DIRECTV BROADBAND INC']
doc 10488 https://www.sec.gov/Archives/edgar/data/1102448/000089161800001777/
doc 10836 https://www.sec.gov/Archives/edgar/data/1102448/000089161800001737/
doc 11553 https://www.sec.gov/Archives/edgar/data/1102448/000089161800001649/
doc 11015 https://www.sec.gov/Archives/edgar/data/1102448/000089161800001586/
doc 12248 https://www.sec.gov/Archives/edgar/data/1102448/000089161800001319/

( 85 / 3665 ) SEA
possible_companies: ['NEW DRAGON ASIA CORP']

( 86 / 3665 ) WBSN
possible_companies: ['WEBSENSE INC']
doc 10310 https://www.sec.gov/Archives/edgar/data/1098277/000093639200000148/
doc 10663 https://www.sec.gov/Archives/edgar/data/1098277/000093639200000146/
doc 13673 https://www.sec.gov/Archives/edgar/data/1098277/000109581100000445/
doc 12079 https://www.sec.gov/Arc

doc 12956 https://www.sec.gov/Archives/edgar/data/1101179/000095011600000191/

( 109 / 3665 ) IMPV
IMPV data already exists, skipping...

( 110 / 3665 ) ACTI
possible_companies: ['ACTIVCARD SA']

( 111 / 3665 ) ASIQ
possible_companies: ['AS-IP TECH INC']

( 112 / 3665 ) LOUD
possible_companies: ['LOUDEYE CORP']

( 113 / 3665 ) SHOC
possible_companies: ['LENCO MOBILE INC.']

( 114 / 3665 ) RVSN
possible_companies: ['RADVISION LTD']

( 115 / 3665 ) FAIM
possible_companies: []
no possible companies: FAIM

( 116 / 3665 ) DTAS
possible_companies: ['DIGITAS INC']
doc 10308 https://www.sec.gov/Archives/edgar/data/1100885/000092701600000815/
doc 10310 https://www.sec.gov/Archives/edgar/data/1100885/000092701600000800/
doc 10487 https://www.sec.gov/Archives/edgar/data/1100885/000092701600000697/
doc 11190 https://www.sec.gov/Archives/edgar/data/1100885/000092701600000664/
doc 16850 https://www.sec.gov/Archives/edgar/data/1100885/000092701600000448/

( 117 / 3665 ) HOMG
possible_companies: ['HOM

...

doc 37540 https://www.sec.gov/Archives/edgar/data/1027401/
doc 37540 https://www.sec.gov/Archives/edgar/data/1027401/
doc 37540 https://www.sec.gov/Archives/edgar/data/1027401/
doc 37540 https://www.sec.gov/Archives/edgar/data/1027401/

( 3303 / 3665 ) MSDI
MSDI data already exists, skipping...

( 3304 / 3665 ) MIIIU
MIIIU data already exists, skipping...

( 3305 / 3665 ) STLRU
STLRU data already exists, skipping...

( 3306 / 3665 ) AIRG
AIRG data already exists, skipping...

( 3307 / 3665 ) MEDP
MEDP data already exists, skipping...

( 3308 / 3665 ) PTGX
PTGX data already exists, skipping...

( 3309 / 3665 ) ATMR
ATMR data already exists, skipping...

( 3310 / 3665 ) GEMP
GEMP data already exists, skipping...

( 3311 / 3665 ) FHB
FHB data already exists, skipping...

( 3312 / 3665 ) NTNX
NTNX data already exists, skipping...

( 3313 / 3665 ) TRHC
TRHC data already exists, skipping...

( 3314 / 3665 ) FLGT
FLGT data already exists, skipping...

( 3315 / 3665 ) TYHT
TYHT data already ex


( 3395 / 3665 ) ZYME
possible_companies: ['ZYMEWORKS INC.']

( 3396 / 3665 ) VRNA
possible_companies: ['VERONA PHARMA PLC']

( 3397 / 3665 ) FND
FND data already exists, skipping...

( 3398 / 3665 ) CATS
CATS data already exists, skipping...

( 3399 / 3665 ) WTTR
WTTR data already exists, skipping...

( 3400 / 3665 ) TOCA
TOCA data already exists, skipping...

( 3401 / 3665 ) CADE
CADE data already exists, skipping...

( 3402 / 3665 ) HCC
HCC data already exists, skipping...

( 3403 / 3665 ) YEXT
YEXT data already exists, skipping...

( 3404 / 3665 ) NETS
possible_companies: ['NETSHOES (CAYMAN) LTD.']

( 3405 / 3665 ) VEACU
VEACU data already exists, skipping...

( 3406 / 3665 ) OKTA
OKTA data already exists, skipping...

( 3407 / 3665 ) FMCIU
FMCIU data already exists, skipping...

( 3408 / 3665 ) SNDR
possible_companies: ['SCHNEIDER NATIONAL, INC. VOTING TRUST', 'SCHNEIDER NATIONAL, INC.']

( 3409 / 3665 ) ELVT
ELVT data already exists, skipping...

( 3410 / 3665 ) HESM
HESM data al

possible_companies: ['FAT BRANDS, INC']

( 3507 / 3665 ) LYL
possible_companies: ['DRAGON VICTORY INTERNATIONAL LTD']

( 3508 / 3665 ) MOSCU
MOSCU data already exists, skipping...

( 3509 / 3665 ) REDU
possible_companies: ['BAIN CAPITAL RISE EDUCATION IV CAYMAN LTD', 'RISE EDUCATION CAYMAN LTD']

( 3510 / 3665 ) RMBL
RMBL data already exists, skipping...

( 3511 / 3665 ) SE
possible_companies: ['1794 COMMODORE OVERSEAS FUND LTD', '1794 EVENT DRIVEN OVERSEAS FUND LTD', '3D SEA II LTD', 'ABB ASEA BROWN BOVERI LTD', 'ABINA ASEAN CO., LTD', 'ACM EMERGING MARKETS MULTI STRATEGY (OVERSEAS) FUND I LTD', 'ACORN OVERSEAS LTD.', 'ACORN OVERSEAS LTD', 'ACORN OVERSEAS, LTD.', 'ADVANCED RESEARCH ALPHA FUND, LTD.', 'ADVANCED SEA INTERNATIONAL LTD', 'ADVISORY U S EQUITY MARKET NEUTRAL OVERSEAS FUND LTD', 'ADVISORY U S EQUITY OVERSEAS FUND I LTD', 'ADVISORY US EQUITY OVERSEAS FUND I LTD', 'AFG RELATIVE VALUE OPPORTUNITY OVERSEAS FUND LTD', 'AIG DKR SOUNDSHORE OVERSEAS INVESTORS LTD', 'ALL SEAS INVESTO


( 3512 / 3665 ) MDB
MDB data already exists, skipping...

( 3513 / 3665 ) QD
possible_companies: ['QUDIAN INC.']

( 3514 / 3665 ) OPTN
OPTN data already exists, skipping...

( 3515 / 3665 ) KIDS
KIDS data already exists, skipping...

( 3516 / 3665 ) HAIR
HAIR data already exists, skipping...

( 3517 / 3665 ) CARG
CARG data already exists, skipping...

( 3518 / 3665 ) SWCH
possible_companies: ['CLOUDSWITCH, INC.', 'NETSWITCH, INC.', 'POWER2SWITCH, INC.', 'SPEECHSWITCH, INC.', 'SWITCH, INC.', 'THREATSWITCH, INC.']

( 3519 / 3665 ) RYTM
RYTM data already exists, skipping...

( 3520 / 3665 ) BRACU
BRACU data already exists, skipping...

( 3521 / 3665 ) BOXL
BOXL data already exists, skipping...

( 3522 / 3665 ) RETO
possible_companies: ['RETO ECO-SOLUTIONS, INC.']

( 3523 / 3665 ) BRPAU
BRPAU data already exists, skipping...

( 3524 / 3665 ) BXG
BXG data already exists, skipping...

( 3525 / 3665 ) AMRH
AMRH data already exists, skipping...

( 3526 / 3665 ) SAIL
SAIL data already exists, 

possible_companies: ['GOLDEN BULL LTD']

( 3615 / 3665 ) AIHS
AIHS data already exists, skipping...

( 3616 / 3665 ) TIBRU
TIBRU data already exists, skipping...

( 3617 / 3665 ) ZS
ZS data already exists, skipping...

( 3618 / 3665 ) RCUS
RCUS data already exists, skipping...

( 3619 / 3665 ) BWB
BWB data already exists, skipping...

( 3620 / 3665 ) OPESU
OPESU data already exists, skipping...

( 3621 / 3665 ) BTAI
BTAI data already exists, skipping...

( 3622 / 3665 ) DOCU
DOCU data already exists, skipping...

( 3623 / 3665 ) GSHD
GSHD data already exists, skipping...

( 3624 / 3665 ) SMAR
SMAR data already exists, skipping...

( 3625 / 3665 ) LASR
LASR data already exists, skipping...

( 3626 / 3665 ) CDAY
CDAY data already exists, skipping...

( 3627 / 3665 ) LEVL
LEVL data already exists, skipping...

( 3628 / 3665 ) PVTL
PVTL data already exists, skipping...

( 3629 / 3665 ) SURF
SURF data already exists, skipping...

( 3630 / 3665 ) MOR
possible_companies: ['MORPHOSYS AG']

( 3